Adam Duke

adamduke@uchicago.edu

ADSP 32013 | 2024 Q3

University of Chicago

# Intro Julia + JuMP

#### Dependencies

In [ ]:
# utility modules
using Logging

# optimizaiton modules
import JuMP
import GLPK
import HiGHS
import Gurobi

# local modules
include("modules/ADSP32013/include.jl")
import .ADSP32013

#### Global

Activate Gurobi License

In [ ]:
if !isdefined(Main, :GRB_ENV)
    try
        global GRB_ENV = Gurobi.Env()
        @info("Gurobi license activated")
    catch e
        @warn("Gurobi license unavailable: ", e)
    end
else
    @info("Gurobi license already activated")
end

Establish Console Logger

In [ ]:
ConsoleLogger(stdout, Logging.Info) |> global_logger

# NullLogger() |> global_logger

# io = open("log.txt", "w+")
# SimpleLogger(io, Logging.Debug) |> global_logger
# flush(io)
# close(io)

#### Problem Statement
$$
\begin{align}
    \max\ & x + y \\
    s.t.\ & x + 2y \le 6 \\
    & 3x + y \le 8 \\
    & x \ge 0 \\
    & y \in [0,2.5]
\end{align}
$$

https://www.geogebra.org/graphing/qwwmsvad

### Approaches

#### 1. Naive

In [ ]:
model = JuMP.Model(()->GLPK.Optimizer())
# model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))

JuMP.@variable(model, x)
JuMP.@variable(model, y)

JuMP.@constraint(model, x + 2y <= 6)
JuMP.@constraint(model, 3x + y <= 8)
JuMP.@constraint(model, x >= 0)
JuMP.@constraint(model, 0 <= y <= 2.5)

JuMP.@objective(model, Max, x + y)

JuMP.optimize!(model)

print(JuMP.objective_value(model))

#### 2. Verbose

In [ ]:
model = JuMP.Model(()->GLPK.Optimizer())
# model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))

JuMP.@variable(model, x)
JuMP.@variable(model, y)

JuMP.@constraint(model, x + 2y <= 6)
JuMP.@constraint(model, 3x + y <= 8)
JuMP.@constraint(model, x >= 0)
JuMP.@constraint(model, 0 <= y <= 2.5)

JuMP.@objective(model, Max, x + y)

@debug(model)

JuMP.optimize!(model)

if JuMP.is_solved_and_feasible(model, allow_local=false, dual=true)
    @debug("summary", JuMP.solution_summary(model))
    @info("optimum", JuMP.objective_value(model))
    @info("solution", round.(JuMP.value.(JuMP.all_variables(model)), digits=4))
else
    @error(JuMP.termination_status(model))
end

#### 3. Concise

In [ ]:
model = JuMP.Model(()->GLPK.Optimizer())
# model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))

JuMP.@variables(model, begin
    x >= 0
    0 <= y <= 2.5
end)

JuMP.@constraints(model, begin
    x + 2y <= 6
    3x + y <= 8
end)

JuMP.@objective(model, Max, x + y)

@debug(model)

JuMP.optimize!(model)

ADSP32013.report(model)

#### 4. Succinct
Original
$$
\begin{align}
    max&\
    c^Tx &&\equiv
    \begin{bmatrix}
        1 \\
        1
    \end{bmatrix}^T
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} \\
    s.t.&\ Ax \le b &&\equiv
    \begin{bmatrix}
        1 & 2 \\
        3 & 1 \\
    \end{bmatrix}
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix}
    \le
    \begin{bmatrix}
        6 \\
        8 \\
    \end{bmatrix} \\
    & x_1 \ge 0 \\
    & x_2 \in [0,2.5]
\end{align}
$$

Standardized
$$
\begin{align}
    min&\
    c^Tx &&\equiv
    \begin{bmatrix}
        -1 \\
        -1
    \end{bmatrix}^T
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix} \\
    s.t.&\ Ax \le b &&\equiv
    \begin{bmatrix}
        1 & 2 \\
        3 & 1 \\
        0 & 1
    \end{bmatrix}
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix}
    \le
    \begin{bmatrix}
        6 \\
        8 \\
        2.5
    \end{bmatrix}\\
    & x \ge 0 \\
\end{align}
$$

In [ ]:
A = [1 2; 3 1; 0 1]
b = [6; 8; 2.5]
c = [-1; -1]

ADSP32013.standard_setup(A, b, c) |> ADSP32013.report
# ADSP32013.standard_setup(A, b, c, ()->Gurobi.Optimizer(GRB_ENV)) |> ADSP32013.report

Preprocessed
$$
\begin{align}
    min&\
    c^Tx &&\equiv
    \begin{bmatrix}
        -1 \\
        -1 \\
        0 \\
        0 \\
        0
    \end{bmatrix}^T
    \begin{bmatrix}
        x_1 \\
        x_2 \\
        s_1 \\
        s_2 \\
        s_3
    \end{bmatrix} \\
    s.t.&\ Ax = b &&\equiv
    \begin{bmatrix}
        1 & 2 & 1 & 0 & 0 \\
        3 & 1 & 0 & 1 & 0 \\
        0 & 1 & 0 & 0 & 1
    \end{bmatrix}
    \begin{bmatrix}
        x_1 \\
        x_2 \\
        s_1 \\
        s_2 \\
        s_3
    \end{bmatrix}
    =
    \begin{bmatrix}
        6 \\
        8 \\
        2.5
    \end{bmatrix}\\
    & x \ge 0 \\
    & s \ge 0
\end{align}
$$

In [ ]:
A = [
        1 2 1 0 0; 
        3 1 0 1 0; 
        0 1 0 0 1;
    ]
b = [6; 8; 2.5]
c = [-1; -1; 0; 0; 0]

ADSP32013.standard_setup(A, b, c) |> ADSP32013.report
# ADSP32013.standard_setup(A, b, c, ()->Gurobi.Optimizer(GRB_ENV)) |> ADSP32013.report